# DM-27813 Crowded Fields to Zooniverse

In [1]:
import os
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, ImageNormalize)

import plotLightcurve
import apdbPlots

First we want to get a random subset of DIASources from the APDB.

In [2]:
REPO = '/project/sullivan/saha/ap_pipe/cw/'
parent = "/project/sullivan/saha/cwcoadds_processed"

In [3]:
connection = sqlite3.connect(f'{REPO}/association.db')

In [4]:
butler = dafPersist.Butler(inputs=[REPO, parent])

In [5]:
%%time 
# to get a random sample: ORDER BY RANDOM() 
ds_sample = pd.read_sql_query('select * from DiaSource LIMIT 10;', connection)

CPU times: user 10.4 ms, sys: 1.18 ms, total: 11.6 ms
Wall time: 9.69 ms


Now we want to get calexp, template, and difference cutouts for each DIASource.

In [6]:
apdbPlots.addVisitCcdToSrcTable(ds_sample)

,diaSourceId,ccdVisitId,diaObjectId,ssObjectId,parentDiaSourceId,prv_procOrder,ssObjectReassocTime,midPointTai,ra,raErr,...,extendedness,spuriousness,flags,pixelId,filterName,filterId,isDipole,bboxSize,ccd,visit
0,184157035032479591,42877401,184157035032479591,0,0,0,None,57116.286309,270.371311,None,...,None,None,25299404,0,g,1,0,33,01,428774
1,184157035032479592,42877401,184157035032479592,0,0,0,None,57116.286309,270.370908,None,...,None,None,31066572,0,g,1,0,36,01,428774
2,184157035032479593,42877401,184157035032479593,0,0,0,None,57116.286309,270.372414,None,...,None,None,25168328,0,g,1,0,48,01,428774
3,184157035032479594,42877401,184157035032479594,0,0,0,None,57116.286309,270.370768,None,...,None,None,31066600,0,g,1,0,26,01,428774
4,184157035032479595,42877401,184157035032479595,0,0,0,None,57116.286309,270.370701,None,...,None,None,31066568,0,g,1,0,17,01,428774
5,184157035032479596,42877401,184157035032479596,0,0,0,None,57116.286309,270.370604,None,...,None,None,31066568,0,g,1,0,26,01,428774
6,184157035032479597,42877401,184157035032479597,0,0,0,None,57116.286309,270.370389,None,...,None,None,31066312,0,g,1,0,24,01,428774
7,184157035032479598,42877401,184157035032479598,0,0,0,None,57116.286309,270.370290,None,...,None,None,31066312,0,g,1,1,16,01,428774
8,184157035032479599,42877401,184157035032479599,0,0,0,None,57116.286309,270.370666,None,...,None,None,25299144,0,g,1,0,25,01,428774
9,184157035032479600,42877401,184157035032479600,0,0,0,None,57116.286309,270.369189,None,...,None,None,31164872,0,g,1,0,30,01,428774


In [20]:
def get_template_cutout(butler, scienceImage, centerSource, size=lsst.geom.Extent2I(30, 30), filter='g'):
    skyMap = butler.get(datasetType='deepCoadd_skyMap')
    expWcs = scienceImage.getWcs()
    expBoxD = lsst.geom.Box2D(scienceImage.getBBox())
    expBoxD.grow(10)
    ctrSkyPos = expWcs.pixelToSky(expBoxD.getCenter())
    tractInfo = skyMap.findTract(ctrSkyPos)
    skyCorners = [expWcs.pixelToSky(pixPos) for pixPos in expBoxD.getCorners()]
    patchList = tractInfo.findPatchList(skyCorners)
    print(f"trying {len(patchList)} patches")
    for patchInfo in patchList:
        templateDataId = dict(
            tract=tractInfo.getId(),
            patch="%s,%s" % (patchInfo.getIndex()[1], patchInfo.getIndex()[0]),
            filter=filter,
        )
        try:
            coaddTemplate = butler.get('deepCoadd', dataId=templateDataId)
        except dafPersist.butlerExceptions.NoResults:
            template = None
            continue
        else:
            print(f'trying cutout {centerSource} in {skyCorners}')
            try:
                from IPython.core.debugger import set_trace
                set_trace()
                template = coaddTemplate.getCutout(centerSource, size)
            except lsst.pex.exceptions.InvalidParameterError:
                template = None
                continue
            else:
                # Only loop through patches until you find one containing the source
                break

    if template is not None:
        print(f"Template dataId {templateDataId}")
    else:
        print(f"No template found for {templateDataId}")
    return template

In [21]:
def get_cutouts(dataId, center, size=lsst.geom.Extent2I(30, 30)):
    exposure = butler.get('calexp', dataId).getCutout(center, size)
    difference = butler.get('deepDiff_differenceExp', dataId)
    template = get_template_cutout(butler, exposure, center, filter='g')
    return exposure, difference, template

In [26]:
dataId={'ccd': int(ds_sample['ccd'][0]), 'visit': int(ds_sample['visit'][0])}
center=lsst.geom.SpherePoint(ds_sample['ra'][0], ds_sample['decl'][0], lsst.geom.degrees)

In [ ]:
get_cutouts(dataId, center)

trying 1 patches
trying cutout (270.371311, -29.052358) in [SpherePoint(270.3692063087432*geom.degrees, -29.05048396076056*geom.degrees), SpherePoint(270.36917392158267*geom.degrees, -29.054133220377047*geom.degrees), SpherePoint(270.373350019409*geom.degrees, -29.05416039107873*geom.degrees), SpherePoint(270.37338229862104*geom.degrees, -29.050511194540984*geom.degrees)]
> <ipython-input-20-fd8ef9ca9553>(27)get_template_cutout()
     25                 from IPython.core.debugger import set_trace
     26                 set_trace()
---> 27                 template = coaddTemplate.getCutout(centerSource, size)
     28             except lsst.pex.exceptions.InvalidParameterError:
     29                 template = None



ipdb>  patchList


(PatchInfo(index=(10, 12), innerBBox=(minimum=(40000, 48000), maximum=(43999, 51999)), outerBBox=(minimum=(39900, 47900), maximum=(44099, 52099))),)


ipdb>  n


lsst.pex.exceptions.wrappers.InvalidParameterError: 
  File "src/image/Exposure.cc", line 221, in lsst::afw::image::Exposure<ImagePixelT, MaskPixelT, VariancePixelT> lsst::afw::image::Exposure<ImageT, MaskT, VarianceT>::getCutout(const lsst::geom::SpherePoint&, const Extent2I&) const [with ImageT = float; MaskT = int; VarianceT = float; lsst::geom::Extent2I = lsst::geom::Extent<int, 2>]
    Point (270.371311, -29.052358) lies at pixel (84623.2, 70576.3), which lies outside Exposure Box2I(Point2I(47900, 39900), Extent2I(4200, 4200)) {0}
lsst::pex::exceptions::InvalidParameterError: 'Point (270.371311, -29.052358) lies at pixel (84623.2, 70576.3), which lies outside Exposure Box2I(Point2I(47900, 39900), Extent2I(4200, 4200))'
> <ipython-input-20-fd8ef9ca9553>(27)get_template_cutout()
     25                 from IPython.core.debugger import set_trace
     26                 set_trace()
---> 27                 template = coaddTemplate.getCutout(centerSource, size)
     28             exce

ipdb>  templateDataId


{'tract': 0, 'patch': '12,10', 'filter': 'g'}


ipdb>  templateDataId['patch']='10,12'
ipdb>  butler.get('deepCoadd', dataId=templateDataId)


*** lsst.daf.persistence.butlerExceptions.NoResults: No locations for get: datasetType:deepCoadd dataId:DataId(initialdata={'tract': 0, 'patch': '10,12', 'filter': 'g'}, tag=set())


In [ ]:
for index,source in ds_sample.iterrows():
    center = lsst.geom.SpherePoint(source['ra'], source['decl'], lsst.geom.degrees)
    dataId = {'ccd': int(source['ccd']), 'visit': int(source['visit'])}
    print(dataId, center)
    exposure, difference, template = get_cutouts(dataId, center)